In [108]:
import urllib.request as ulib
import sys
from functools import reduce

In [14]:
BLOCK_SIZE = 16; # AES clock size is 16 bytes

In [101]:
def xor_bytes(a, *arguments):
    if len(arguments) == 0:
        return a
    arg = arguments[0]
    assert(len(a) == len(arg))
    a = bytes([b1 ^ b2 for b1, b2 in zip(a, arg)])
    return xor_bytes(a, *arguments[1:])

def xor_byte_n(byteobj, n, bt):
    return bytes([
        byteobj[i] ^ bt if i == n else byteobj[i]
        for i in range(len(byteobj))
    ])

def xor_bytes_after_n(byteobj, n, bt):
    return bytes([
        byteobj[i] ^ bt if i >= n else byteobj[i]
        for i in range(len(byteobj))
    ])

def replace_byte_n(byteobj, n, bt):
    return bytes([
        bt if i == n else byteobj[i]
        for i in range(len(byteobj))
    ])

In [183]:
url_base = 'http://crypto-class.appspot.com/po?er='

def query(q) -> bool:
    target = url_base + ulib.quote(q)
    req = ulib.Request(target)
    try:
        f = ulib.urlopen(req)
    except ulib.HTTPError as e:
        if e.code == 404:
            return True # gotcha!
        return False
    print('Response success')
    return True

#### Prepare cipher text blocks

In [184]:
ctext = 'f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4'
bts = bytes.fromhex(ctext)
nblocks = len(bts) // BLOCK_SIZE

assert(len(bts) == nblocks * BLOCK_SIZE)
blocks = [bts[k:k+BLOCK_SIZE] for k in range(0, len(bts), BLOCK_SIZE)]

In [196]:
ASCII_SEQ \
= list(range(97, 123)) \
+ list(range(65, 97)) \
+ list(range(0, 65)) \
+ list(range(123,256))

def guess_block_n(blocks, n):
    assert(n >= 1) # zero block is an IV
    assert(n < len(blocks))
    base = reduce(lambda cum, e: cum + e, blocks[:n-1], bytes(0))
    worker = blocks[n-1]
    cur_block = blocks[n]
    result = bytes(BLOCK_SIZE)
    for byte_offset in range(BLOCK_SIZE-1, -1, -1):
        # xor with valid padding
        pad = BLOCK_SIZE-byte_offset
        padded = xor_bytes_after_n(worker, byte_offset, pad)
        # a hack to avoid wrong paddings
        padded = xor_byte_n(padded, byte_offset-1, 97)
        g = 0
        for i_g in range(256):
            g = ASCII_SEQ[i_g]
            hack = xor_byte_n(padded, byte_offset, g)
            Q = hack + cur_block
            if query(Q.hex()):
                print('\r'+(' '*30)+'\r', end='')
                print('Sent: %d queries' % (i_g+1), end='')
                print('\nByte<%d>: %d' % (byte_offset, g))
                break
            if i_g % 5 == 0:
                print('\r'+(' '*30)+'\r', end='')
                print('Sent: %d queries' % i_g, end='')
            assert i_g < 255, "didn't manage to guess the byte"
        worker = xor_byte_n(worker, byte_offset, g)
        result = replace_byte_n(result, byte_offset, g)
    return result

In [198]:
msg = bytes(0)
for n in range(1, nblocks):
    print('==================')
    print('Guessing block %d' % n)
    print('==================')
    msg += guess_block_n(blocks, n)
    
Message = reduce(lambda cum, e: cum + chr(e), list(msg), '')
print('\n<<<<<<<<<<<<<<<<<<')
print(Message)
print('\n>>>>>>>>>>>>>>>>>>')

Guessing block 1
Sent: 91 queries              
Byte<15>: 32
Sent: 19 queries              
Byte<14>: 115
Sent: 4 queries               
Byte<13>: 100
Sent: 18 queries              
Byte<12>: 114
Sent: 15 queries              
Byte<11>: 111
Sent: 49 queries              
Byte<10>: 87
Sent: 91 queries              
Byte<9>: 32
Sent: 3 queries               
Byte<8>: 99
Sent: 9 queries               
Byte<7>: 105
Sent: 7 queries               
Byte<6>: 103
Sent: 1 queries               
Byte<5>: 97
Sent: 39 queries              
Byte<4>: 77
Sent: 91 queries              
Byte<3>: 32
Sent: 5 queries               
Byte<2>: 101
Sent: 8 queries               
Byte<1>: 104
Sent: 46 queries              
Byte<0>: 84
Guessing block 2
Sent: 19 queries              
Byte<15>: 115
Sent: 41 queries              
Byte<14>: 79
Sent: 91 queries              
Byte<13>: 32
Sent: 8 queries               
Byte<12>: 104
Sent: 19 queries              
Byte<11>: 115
Sent: 9 queries               
Byte<10>: 